In [ ]:
"""

Examples on how TFLite converts following models to TFLite models:
1. tf.keras models   2. Saved Models  3. Concrete functions

TFLite models are called as TFLite Flatbuffer. TFLite models is a format suitable for TFLite Interpreter to
run our models on these devices.

"""

#imports
import tensorflow as tf
import pathlib

In [ ]:
#data
x = [-1,0,1,2,3,4]
y = [-3,-1,1,3,5,7]

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])
])

model.compile(optimizer = 'sgd', loss = 'mean_squared_error')

model.fit(x, y, epochs=10)

In [ ]:
# ----------Saved model------------ #

# Export Path(Temporary path, won't run)
export_path = '/tmp/saved_model'
tf.saved_model.save(model, export_path)

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_path)
tflite_model = converter.convert()

# Save TFLite model. Extension must be .tflite
tflite_model_file = pathlib.Path('tmp/foo.tflite')
tflite_model_file.write_bytes(tflite_model)

In [ ]:
# ----------Keras model------------ #

model = tf.keras.applications.MobileNet(weights='imagenet',input_shape = [224,224,3])

# Model can be passed directly to converter, no need to save it 
converter = tf.lite.TFLiteConverter.from_keras.model(model)
tf_lite_model = converter.convert()

tf_lite_model_file = pathlib.Path('tmp/foo.tflite')
tf_lite_model_file.write_bytes(tf_lite_model)

In [ ]:
# ----------Concrete Function------------ #

model = tf.keras.applications.MobileNet(weights='imagenet', input_shape=[224,224,3])

run_model = tf.function(lambda x: model(x))

concrete_function = run_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape,
                                                                  model.input[0].dtype))

converter = tf.lite.TFLiteConverter.from_concrete_function([concrete_function])
tflite_model = converter.convert()

In [ ]:
# Saving with command-line - Saved Model
tflite_convert --output_file=model.tflite --saved_model_dir="tmp/saved_model"

# Saving with command-line - Keras Model
tflite_convert --output_file=model.tflite --keras_model_file=model.h5